<a href="https://colab.research.google.com/github/fernandoczen/projeto-AREE-ebanx/blob/main/projeto_AREE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Instalando dependências

In [ ]:
!pip install basedosdados -q
!pip install basedosdados[upload] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires tomlkit<0.14.0,>=0.12.0, but you have tomlkit 0.11.8 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
import basedosdados as bd
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True);

Mounted at /content/drive


# 2 - Autenticação

In [ ]:
from google.colab import auth
from google.cloud import bigquery
project_id = 'ebanx-analise-renda-evasao';

# Autentica o usuário
auth.authenticate_user(clear_output = True, project_id= project_id);
client = bigquery.Client(project=project_id);

MessageError: Error: credential propagation was unsuccessful

## 3 - Importando base de dados
 Para melhor lidar com os dados, a abordagem preferida foi a de coletar diferentes tabelas e fazer um merge com todos os dados importantes para a nossa pesquisa.

In [ ]:
## Base de dados referentes a dados de renda.

database_renda_pnad = bd.read_sql(''' WITH
dicionario_V1022 AS (
    SELECT
        chave AS chave_V1022,
        valor AS descricao_V1022
    FROM `basedosdados.br_ibge_pnadc.dicionario`
    WHERE
        TRUE
        AND nome_coluna = 'V1022'
        AND id_tabela = 'rendimentos_outras_fontes'
),
dicionario_V1023 AS (
    SELECT
        chave AS chave_V1023,
        valor AS descricao_V1023
    FROM `basedosdados.br_ibge_pnadc.dicionario`
    WHERE
        TRUE
        AND nome_coluna = 'V1023'
        AND id_tabela = 'rendimentos_outras_fontes'
),
dicionario_VD5006 AS (
    SELECT
        chave AS chave_VD5006,
        valor AS descricao_VD5006
    FROM `basedosdados.br_ibge_pnadc.dicionario`
    WHERE
        TRUE
        AND nome_coluna = 'VD5006'
        AND id_tabela = 'rendimentos_outras_fontes'
)
SELECT
    dados.ano as ano,
    dados.id_domicilio as id_domicilio,
    dados.id_pessoa as id_pessoa,
    descricao_V1022 AS V1022,
    descricao_V1023 AS V1023,
    dados.V2001 as V2001,
    descricao_VD5006 AS VD5006
FROM `basedosdados.br_ibge_pnadc.rendimentos_outras_fontes` AS dados
LEFT JOIN `dicionario_V1022`
    ON dados.V1022 = chave_V1022
LEFT JOIN `dicionario_V1023`
    ON dados.V1023 = chave_V1023
LEFT JOIN `dicionario_VD5006`
    ON dados.VD5006 = chave_VD5006''');

Downloading:  70%|██████▉   |

KeyboardInterrupt: 

In [ ]:
## Base de dados referente a dados educacionais.

database_educacao_pnad = bd.read_sql (''' SELECT
    dados.id_pessoa as id_pessoa,
    dados.V3002 as V3002,
    dados.V3002A as V3002A,
    dados.V3006 as V3006,
    dados.V3008 as V3008,
    dados.V3034 as V3034,
    dados.V3034A as V3034A
FROM `basedosdados.br_ibge_pnadc.educacao` AS dados ''')

In [ ]:
## Base de dados referente a dados pessoais.

database_pessoal_pnad = bd.read_sql('''WITH
dicionario_V2007 AS (
    SELECT
        chave AS chave_V2007,
        valor AS descricao_V2007
    FROM `basedosdados.br_ibge_pnadc.dicionario`
    WHERE
        TRUE
        AND nome_coluna = 'V2007'
        AND id_tabela = 'microdados'
),
dicionario_V2010 AS (
    SELECT
        chave AS chave_V2010,
        valor AS descricao_V2010
    FROM `basedosdados.br_ibge_pnadc.dicionario`
    WHERE
        TRUE
        AND nome_coluna = 'V2010'
        AND id_tabela = 'microdados'
)
SELECT
    dados.id_pessoa as id_pessoa,
    descricao_V2007 AS V2007,
    dados.V2009 as V2009,
    descricao_V2010 AS V2010
FROM `basedosdados.br_ibge_pnadc.microdados` AS dados
LEFT JOIN `dicionario_V2007`
    ON dados.V2007 = chave_V2007
LEFT JOIN `dicionario_V2010`
    ON dados.V2010 = chave_V2010''')

# 4 - Tratamento de tabelas
 Nessa seção, utiliza-se principalmente o Pandas para fazer o tratamento de tabelas.

In [ ]:
database_renda_pnad.info()
database_renda_pnad.head()
database_renda_pnad = database_renda_pnad.rename(columns={
    'V1022':'situacao_domicilio',
    'V1023': 'tipo_de_area',
    'V2001':'n_pessoas_domicilio',
    'VD5006':'faixa_salarial'});
database_renda_pnad.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429834 entries, 0 to 1429833
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   ano           1429834 non-null  Int64 
 1   id_domicilio  1429834 non-null  object
 2   id_pessoa     1429834 non-null  object
 3   V1022         1429834 non-null  object
 4   V1023         1429834 non-null  object
 5   V2001         1429834 non-null  Int64 
 6   VD5006        942719 non-null   object
dtypes: Int64(2), object(5)
memory usage: 79.1+ MB


,ano,id_domicilio,id_pessoa,situacao_domicilio,tipo_de_area,n_pessoas_domicilio,faixa_salarial
0,2022,1700248400610,170024840061003,Urbana,Capital,4,Mais de 2 até 3 salários mínimos
1,2022,1700248400610,170024840061001,Urbana,Capital,4,Mais de 2 até 3 salários mínimos
2,2022,1700248400610,170024840061004,Urbana,Capital,4,Mais de 2 até 3 salários mínimos
3,2022,1700248400610,170024840061002,Urbana,Capital,4,Mais de 2 até 3 salários mínimos
4,2022,1700248400710,170024840071004,Urbana,Capital,5,Mais de 5 salários mínimos


In [ ]:
database_educacao_pnad.info()
database_educacao_pnad.head()
database_educacao_pnad = database_educacao_pnad.rename(columns={
    'V3002':'frequenta_escola',
    'V3002A':'tipo_escola',
    'V3006':'serie',
    'V3008':'frequentou_escola',
    'V3034':'motivo_nao_frequenta',
    'V3034A':'idade_frequentou'
})
database_educacao_pnad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707493 entries, 0 to 707492
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id_pessoa  707493 non-null  object
 1   V3002      707493 non-null  object
 2   V3002A     202417 non-null  object
 3   V3006      169476 non-null  object
 4   V3008      505076 non-null  object
 5   V3034      54101 non-null   object
 6   V3034A     54291 non-null   object
dtypes: object(7)
memory usage: 37.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707493 entries, 0 to 707492
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id_pessoa             707493 non-null  object
 1   frequenta_escola      707493 non-null  object
 2   tipo_escola           202417 non-null  object
 3   serie                 169476 non-null  object
 4   frequentou_escola     505076 non-null  object
 5   motivo_nao_freque

In [ ]:
database_pessoal_pnad = database_pessoal_pnad.rename(columns={
    'V2007':'sexo',
    'V2009':'idade',
    'V2010':'cor_raca'})
database_pessoal_pnad.head()

,id_pessoa,sexo,idade,cor_raca
0,420063814090101,Homem,76,Branca
1,420107125050101,Homem,75,Branca
2,420011034010101,Homem,77,Branca
3,420032428030101,Homem,79,Branca
4,420079343120101,Homem,76,Parda


In [ ]:
database_final = pd.merge(database_pessoal_pnad, database_educacao_pnad, on='id_pessoa', how='inner');
database_final = pd.merge(database_final, database_renda_pnad, on='id_pessoa', how='inner');


In [ ]:
database_final = database_final.drop(columns={
    'id_domicilio',
    'tipo_escola' })

In [ ]:
### database_final = database_final.drop(columns={
##    'id_domicilio',
#    'tipo_escola' })
database_final_EM = database_final[database_final['idade'] > 14];
database_final_EM = database_final_EM[database_final_EM['idade'] < 19];

In [ ]:
database_final_EM.head()

NameError: name 'database_final_EM' is not defined

# 5 - Exportação como CSV

In [ ]:
database_final_EM.to_csv("dataset_completo_EM.csv")

NameError: name 'database_final_EM' is not defined

# 6 - Tabelas filtradas

In [ ]:
dataset_original = pd.read_csv("/content/drive/MyDrive/EBANX-AREE/dataset_completo_EM (1).csv");
dataset_original.loc[dataset_original['frequenta_escola'] == 2, 'frequenta_escola'] = 0
dataset_original.loc[dataset_original['frequentou_escola'] == 2, 'frequentou_escola'] = 0
dataset_original.head()

## Supomos que, por haver uma faixa de idade, a 1 série é do 1 ano do ensino médio, por exemplo.

,Unnamed: 0,id_pessoa,sexo,idade,cor_raca,frequenta_escola,serie,frequentou_escola,motivo_nao_frequenta,idade_frequentou,ano,situacao_domicilio,tipo_de_area,n_pessoas_domicilio,faixa_salarial
0,54507,430207392081003,Homem,15,Parda,1,9.0,NaN,NaN,NaN,2022,Rural,"Resto da RM (Região Metropolitana, excluindo a...",3,Mais de 1 até 2 salários mínimos
1,54508,310411126141005,Homem,15,Branca,0,NaN,0.0,NaN,NaN,2022,Rural,"Resto da UF (Unidade da Federação, excluindo ...",5,"Mais de 0,25 até 0,5 salário mínimo"
2,54509,310537503011002,Homem,15,Parda,0,NaN,1.0,NaN,NaN,2022,Urbana,"Resto da UF (Unidade da Federação, excluindo ...",4,"Mais de 0,5 até 1 salário mínimo"
3,54510,250067546120703,Mulher,15,Parda,1,1.0,NaN,NaN,NaN,2020,Urbana,Capital,3,Mais de 1 até 2 salários mínimos
4,54511,250067546120703,Mulher,15,Parda,1,1.0,NaN,NaN,NaN,2019,Urbana,Capital,3,Mais de 1 até 2 salários mínimos


In [ ]:
dataset_localizado = pd.DataFrame(
    columns=['Frequenta', 'NFrequenta', 'Total'],
    index=["Classe E", "Classe D", "Classe C+", "Total"]);
dataset_localizado.index.name = "Renda";
dataset_localizado.head()

,Frequenta,NFrequenta,Total
Renda,,,
Classe E,NaN,NaN,NaN
Classe D,NaN,NaN,NaN
Classe C+,NaN,NaN,NaN
Total,NaN,NaN,NaN


In [ ]:
classe_e = ["Até 0,25 salário mínimo", "Mais de 0,25 até 0,5 salário mínimo", "Mais de 0,5 até 1 salário mínimo"];
print(classe_e);
dataset_localizado.loc['Classe E', 'Frequenta'] = dataset_localizado[dataset_original['faixa_salarial'].isin(classe_e) & dataset_original['frequenta_escola'] == 1];
print(dataset_localizado);

['Até 0,25 salário mínimo', 'Mais de 0,25 até 0,5 salário mínimo', 'Mais de 0,5 até 1 salário mínimo']


NameError: name 'dataset_localizado' is not defined

In [ ]:
'''
Anotações do Fernando: Pretendo preencher a tabela acima usando variáveis com os dados simplificados da planilha original.
Além disso, quero também ver para criar uma outra tabela apenas com aqueles que evadiram com variáveis de idade, série e ano de evasão. A ideia é
localizar um ponto em que a evasão tenha mais força.
Acho que no mais é isso.
'''

'\nAnotações do Fernando: Pretendo preencher a tabela acima usando variáveis com os dados simplificados da planilha original.\nAlém disso, quero também ver para criar uma outra tabela apenas com aqueles que evadiram com variáveis de idade, série e ano de evasão. A ideia é\nlocalizar um ponto em que a evasão tenha mais força.\nAcho que no mais é isso. \n'

# 7 - Visualização de dados